In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime


In [2]:
url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Al_cash'

In [3]:
responce = requests.get(url=url)

In [7]:
len(pd.read_html(responce.text))

/var/folders/hp/79qzs1rx74qgjz43pgfcb1340000gn/T/ipykernel_6809/674423051.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  len(pd.read_html(responce.text))


16

In [5]:
pd.read_html(responce.text)[0]

/var/folders/hp/79qzs1rx74qgjz43pgfcb1340000gn/T/ipykernel_6809/3953017053.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(responce.text)[0]


,date,LME Aluminium Cash-Settlement,LME Aluminium 3-month,LME Aluminium stock
0,06. October 2023,2208.00,2239.00,497650
1,05. October 2023,2195.50,2233.00,500850
2,04. October 2023,2231.00,2271.00,502850
3,03. October 2023,2257.00,2294.00,504850
4,02. October 2023,2286.50,2317.00,486775
...,...,...,...,...
197,09. January 2023,2336.50,2370.00,422075
198,06. January 2023,2242.50,2275.00,428525
199,05. January 2023,2270.00,2304.00,436500
200,04. January 2023,2250.00,2287.00,442425


In [ ]:
:method: GET
:scheme: https
:authority: www.lme.com
:path: /api/trading-data/metal-block?datasourceIds=1b72897c-6a30-480c-a74a-cfc2faaaff16&datasourceIds=fc09fcde-6438-4834-9221-98da9ed54eea&datasourceIds=b411edcc-fc99-4da8-ba18-da82b8a5dad4&datasourceIds=6326cb5f-0770-4ba6-a3b4-871ecf0566f7&datasourceIds=14f25c1d-fd36-459d-a228-e088180bfb75&datasourceIds=1dcd0785-8af8-4df3-aae0-084e9a686654&datasourceIds=4b9fe162-6e23-4566-ba72-6f798c10c605&datasourceIds=e7c48262-7026-401e-8e12-3311783bf629
Cookie: SC_ANALYTICS_GLOBAL_COOKIE=17e39e9fb7284aeb9a3b2acd6e5ef550|True; ai_session=v9gSooxIrzDEwfQN7OI/Hj|1696749471127|1696749498746; _ga=GA1.2.1905386367.1692928530; _ga_K7LTMQKVQ8=GS1.2.1696749471.33.0.1696749474.0.0.0; _gid=GA1.2.2117894617.1696749471; OptanonConsent=isGpcEnabled=0&datestamp=Sun+Oct+08+2023+12%3A17%3A51+GMT%2B0500+(%D0%95%D0%BA%D0%B0%D1%82%D0%B5%D1%80%D0%B8%D0%BD%D0%B1%D1%83%D1%80%D0%B3%2C+%D1%81%D1%82%D0%B0%D0%BD%D0%B4%D0%B0%D1%80%D1%82%D0%BD%D0%BE%D0%B5+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F)&version=6.19.0&isIABGlobal=false&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0003%3A1%2CC0002%3A1%2CC0004%3A1&geolocation=RU%3BSVE&AwaitingReconsent=false; _gat_UA-197696809-1=1; _hjAbsoluteSessionInProgress=0; _hjIncludedInSessionSample_2713621=0; _hjSessionUser_2713621=eyJpZCI6ImNhYTYyMzVlLTg1MTUtNTM4ZC1iZGJmLWVmZTAyYjkwMjFjYiIsImNyZWF0ZWQiOjE2OTI5Mjg1MzA0MTYsImV4aXN0aW5nIjp0cnVlfQ==; _hjSession_2713621=eyJpZCI6IjhjY2Y4ODk2LTE2N2EtNDVlMi05NTQ4LTFjZTRmY2Y3ZDE3YSIsImNyZWF0ZWQiOjE2OTY3NDk0NzEyOTYsImluU2FtcGxlIjpmYWxzZSwic2Vzc2lvbml6ZXJCZXRhRW5hYmxlZCI6ZmFsc2V9; s_cc=true; __cf_bm=bK0szykIHDqZ3F42a2IOhlhrIOMwT0Zjqhh6CFHFgLI-1696749471-0-AeNJunxlLOp9xdyfIfK0OXYRpsuic8ZRh78zHKHl9Qw97yymmMqGhZ4KxJfFOVplvkeNe6eO7SJThwTwA8xBlPo=; ASP.NET_SessionId=p3jr2za1llqckaejozdytzk3; shell#lang=en; AMCV_DD0356406298B0640A495CB8%40AdobeOrg=179643557%7CMCIDTS%7C19638%7CMCMID%7C16992543679285738915009814657013146479%7CvVersion%7C5.5.0; OptanonAlertBoxClosed=2023-08-25T01:55:29.895Z; ai_user=kNc4mu4yDxtpq6FwsztCLs|2023-08-25T01:55:25.093Z
Accept: */*
Sec-Fetch-Site: same-origin
Accept-Language: ru
Accept-Encoding: gzip, deflate, br
Sec-Fetch-Mode: cors
Host: www.lme.com
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15
Referer: https://www.lme.com/Metals/Non-ferrous
Connection: keep-alive
Sec-Fetch-Dest: empty
request-id: |8784e21b66a64b6f87b7282af56ce53c.6c4ce9040eeb41c9
traceparent: 00-8784e21b66a64b6f87b7282af56ce53c-6c4ce9040eeb41c9-01


In [ ]:
https://www.lme.com/api/trading-data/metal-block?datasourceIds=1b72897c-6a30-480c-a74a-cfc2faaaff16&datasourceIds=fc09fcde-6438-4834-9221-98da9ed54eea&datasourceIds=b411edcc-fc99-4da8-ba18-da82b8a5dad4&datasourceIds=6326cb5f-0770-4ba6-a3b4-871ecf0566f7&datasourceIds=14f25c1d-fd36-459d-a228-e088180bfb75&datasourceIds=1dcd0785-8af8-4df3-aae0-084e9a686654&datasourceIds=4b9fe162-6e23-4566-ba72-6f798c10c605&datasourceIds=e7c48262-7026-401e-8e12-3311783bf629

In [10]:
url_2 = 'https://www.lme.com/api/trading-data/metal-block?datasourceIds=1b72897c-6a30-480c-a74a-cfc2faaaff16&datasourceIds=fc09fcde-6438-4834-9221-98da9ed54eea&datasourceIds=b411edcc-fc99-4da8-ba18-da82b8a5dad4&datasourceIds=6326cb5f-0770-4ba6-a3b4-871ecf0566f7&datasourceIds=14f25c1d-fd36-459d-a228-e088180bfb75&datasourceIds=1dcd0785-8af8-4df3-aae0-084e9a686654&datasourceIds=4b9fe162-6e23-4566-ba72-6f798c10c605&datasourceIds=e7c48262-7026-401e-8e12-3311783bf629'

In [11]:
responce_2 = requests.get(url=url_2)

In [13]:
responce_2.text

'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><link href="/cdn-cgi/styles/challenges.css" rel="stylesheet"></head><body class="no-js"><div class="main-wrapper" role="main"><div class="main-content"><noscript><div id="challenge-error-title"><div class="h2"><span class="icon-wrapper"><div class="heading-icon warning-icon"></div></span><span id="challenge-error-text">Enable JavaScript and cookies to continue</span></div></div></noscript></div></div><script>(function(){window._cf_chl_opt={cvId: \'2\',cZone: "www.lme.com",cType: \'managed\',cNounce: \'20533\',cRay: \'812c9e7cfc4904a3\',cHash: \'b03c052885e4265\',cUPMDTk: "\\/api\\/trading-data\\/metal-block?datasourceIds=1b72897c-6a30-480c-a74a-cfc2faaaff16&datasourceIds=fc09fcde-6438-